DATASET GENERATION

In [ ]:
import numpy as np

M = 10
n = 5
T = 100


cache_sizes = np.random.randint(low=1, high=10, size=M)
connection_costs = np.random.rand(M, M)


file_requests = np.random.randint(2, size=(T, M, n))
cache_decisions = np.random.randint(2, size=(T, M, n))


for t in range(1, T):
    connection_costs = np.minimum(np.maximum(connection_costs + np.random.uniform(-0.1, 0.1, size=(M, M)), 0), 1)


ccutoff = 0.5
d2d_neighbors = [
    [j for j in range(M) if connection_costs[i, j] <= ccutoff and connection_costs[i, j] <= connection_costs[i, -1]]
    for i in range(M)
]
theta = np.random.randint(2, size=(M, n))

d2d_cache_hits_per_user = np.array([
    np.sum(file_requests[:, i, :] * (1 - cache_decisions[:, i, :]) * (1 - theta[i, :]) * np.min(connection_costs[i, d2d_neighbors[i]] <= c_cutoff))
    for i in range(M)
])


average_chrd2d = np.mean(d2d_cache_hits_per_user) / (T * n)
print("Average D2D Cache Hit Ratio:", average_chrd2d)


Average D2D Cache Hit Ratio: 0.11220000000000001


RNN

100

In [ ]:

M = 10
T = 100


connection_costs = np.random.rand(T, M, M)


historical_costs = connection_costs[:-1]


timesteps = historical_costs.shape[0]
features = M * M
DX = historical_costs.reshape(timesteps, features, 1)
DY = connection_costs[1:].reshape((T-1), M * M)





In [ ]:

DtrainX_reshaped = DtrainX.reshape((-1, features))
DtestX_reshaped = DtestX.reshape((-1, features))
DtrainY_reshaped = DtrainY.reshape((-1, features))
DtestY_reshaped = DtestY.reshape((-1, features))


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


model = Sequential([
    Dense(64, input_shape=(features,)),
    Dense(features)
])


model.compile(optimizer='adam', loss='mean_squared_error')  # Use appropriate loss function


model.fit(x=DtrainX_reshaped, y=DtrainY_reshaped, epochs=epochs, batch_size=batch_size)


predicted_costs = model.predict(DtestX_reshaped)


loss = tf.reduce_mean(tf.losses.mean_squared_error(DtestY_reshaped, predicted_costs))

print("Loss:", loss)


Epoch 1/10
3/3 [==============================] - 1s 9ms/step - loss: 0.6075
Epoch 2/10
3/3 [==============================] - 0s 5ms/step - loss: 0.4300
Epoch 3/10
3/3 [==============================] - 0s 4ms/step - loss: 0.3290
Epoch 4/10
3/3 [==============================] - 0s 4ms/step - loss: 0.2685
Epoch 5/10
3/3 [==============================] - 0s 4ms/step - loss: 0.2289
Epoch 6/10
3/3 [==============================] - 0s 5ms/step - loss: 0.2014
Epoch 7/10
3/3 [==============================] - 0s 4ms/step - loss: 0.1820
Epoch 8/10
3/3 [==============================] - 0s 5ms/step - loss: 0.1679
Epoch 9/10
3/3 [==============================] - 0s 4ms/step - loss: 0.1581
Epoch 10/10
1/1 [==============================] - 0s 84ms/step
Loss: tf.Tensor(0.15373024, shape=(), dtype=float32)


ATTENTION FRAMWORK

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import MultiHeadAttention, Dense, LayerNormalization, LSTM, Add, Multiply

T = 100
M = 500
n = 200

historical_file_requests = np.random.randint(2, size=(T, M, n))
historical_costs = np.random.rand(T, M, M)


DX_encoder = historical_file_requests


DX_decoder = historical_costs[:-1]


DY_attention = historical_costs[1:]

DX_encoder_attention = DX_encoder.reshape((T * M, n))
DX_decoder_attention = DX_decoder.reshape(((T - 1) * M, M))
DY_attention_reshaped = DY_attention.reshape(((T - 1) * M, M))


class GatedResidualNetwork(tf.keras.layers.Layer):
    def __init__(self, units, dropout_rate=0.1):
        super(GatedResidualNetwork, self).__init__()
        self.dense_layer = Dense(units, activation='elu')
        self.gate_layer = Dense(units, activation='sigmoid')
        self.layer_norm = LayerNormalization(epsilon=1e-6)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)

    def call(self, inputs, training=False):
        x = self.dense_layer(inputs)
        gate = self.gate_layer(inputs)
        x = x * gate
        x = self.layer_norm(x)
        x = self.dropout(x, training=training)
        return x



import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, LayerNormalization, LSTM, Add, Multiply


class AttentionModel(tf.keras.Model):
    def __init__(self, num_heads, d_model, num_layers, units, dropout_rate):
        super(AttentionModel, self).__init__()

        self.lstm_encoder = LSTM(units=d_model, return_sequences=True)
        self.ln_encoder = LayerNormalization(epsilon=1e-6)
        self.grn_encoder = GatedResidualNetwork(units=d_model, dropout_rate=dropout_rate)


        self.lstm_decoder = LSTM(units=d_model, return_sequences=True)
        self.ln_decoder = LayerNormalization(epsilon=1e-6)
        self.grn_decoder = GatedResidualNetwork(units=d_model, dropout_rate=dropout_rate)


        self.multihead_attention = MultiHeadAttention(num_heads=num_heads, key_dim=d_model)


        self.dense_layer = Dense(units=M * n, activation='softmax')

    def call(self, inputs, training=False):
        DX_encoder_attention, DX_decoder_attention = inputs


        encoder_output = self.lstm_encoder(DX_encoder_attention)
        encoder_output = self.ln_encoder(encoder_output)
        encoder_output = self.grn_encoder(encoder_output, training=training)

        decoder_output = self.lstm_decoder(DX_decoder_attention)
        decoder_output = self.ln_decoder(decoder_output)
        decoder_output = self.grn_decoder(decoder_output, training=training)


        attn_output = self.multihead_attention(query=decoder_output, value=encoder_output, key=encoder_output)

        encoder_output_projected = self.dense_projection(encoder_output)
        attn_encoder_mult = attn_output * tf.expand_dims(encoder_output, axis=2)
        added_output = attn_encoder_mult + tf.expand_dims(encoder_output, axis=1)
        norm_output = LayerNormalization(epsilon=1e-6)(added_output)

        attn_decoder_mult = attn_output * tf.expand_dims(decoder_output, axis=2)  # Element-wise multiplication
        added_output2 = attn_decoder_mult + tf.expand_dims(norm_output, axis=1)
        norm_output2 = LayerNormalization(epsilon=1e-6)(added_output2)

        output = self.dense_layer(norm_output2)

        return output



model = AttentionModel(num_heads=4, d_model=64, num_layers=3, units=128, dropout_rate=0.1)


model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

new_decoder_sequence_length = 300
DX_decoder_attention_reshaped = DX_decoder_attention.reshape(((T - 1) * M, new_decoder_sequence_length, M))

new_encoder_sequence_length = 150
DX_encoder_attention_reshaped = DX_encoder_attention.reshape((T * M, new_encoder_sequence_length, 1))

model.fit(x=[DX_encoder_attention_reshaped[0:49500], DX_decoder_attention_reshaped],
          y=DY_attention_reshaped, epochs=epochs, batch_size=batch_size)



ValueError: ignored

In [ ]:

DX_encoder_attention_reshaped = DX_encoder_attention.reshape((Timesteps, M, 1)).astype(np.int64)
DX_decoder_attention_reshaped = DX_decoder_attention.reshape((Timesteps, M, 1)).astype(np.int64)


model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])


model.fit(x=[DX_encoder_attention_reshaped[0:49500], DX_decoder_attention_reshaped], y=DY_attention_reshaped, epochs=epochs, batch_size=batch_size)


ValueError: ignored

UCB1 algorithm

In [ ]:

for t in range(test_samples):
    for i in range(M):
        for j in range(n):

            ucb_values = action_values[i] + ucb_parameter * np.sqrt(np.log(train_samples + t + 1) / action_counts[i])


            action = np.argmax(ucb_values)
            reward = test_d2d_cache_hits_per_user[i, action]


            action_counts[i, action] += 1
            action_values[i, action] += (reward - action_values[i, action]) / action_counts[i, action]

            total_rewards += reward


            regret[t, i] = optimal_action_values[i] - test_d2d_cache_hits_per_user[i, action]


average_regret = np.mean(regret)


loss_ucb1 = total_rewards - np.sum(optimal_action_values) * test_samples * n * M

print("Average Regret (UCB1):", average_regret)
print("Loss (UCB1):", loss_ucb1)


predicted_rewards = np.zeros((test_samples, M, n))
for t in range(test_samples):
    for i in range(M):
        for j in range(n):

            ucb_values = action_values[i] + ucb_parameter * np.sqrt(np.log(train_samples + t + 1) / action_counts[i])


            action = np.argmax(ucb_values)
            predicted_rewards[t, i, j] = action


predicted_loss_ucb1 = np.mean(np.square(test_d2d_cache_hits_per_user - predicted_rewards))

print("Loss for Predicted Rewards (UCB1):", predicted_loss_ucb1)


IndexError: ignored

In [ ]:

reshaped_predicted_rewards = predicted_rewards[:, :, 0]
predicted_loss_ucb1 = np.mean(np.square(test_d2d_cache_hits_per_user - reshaped_predicted_rewards[0:19]))

print("Loss for Predicted Rewards (UCB1):", predicted_loss_ucb1)


Loss for Predicted Rewards (UCB1): 3.8796109046137652


In [ ]:
import numpy as np

M = 10
n = 5
T = 100

train_size = int(0.8 * M)
train_d2d_cache_hits_per_user = d2d_cache_hits_per_user[:train_size]
test_d2d_cache_hits_per_user = d2d_cache_hits_per_user[train_size:]


action_values = np.zeros((M, n))


action_counts = np.zeros((M, n))


optimal_action_values = train_d2d_cache_hits_per_user


epsilon = 0.1


regret = np.zeros((train_size, M))
total_rewards = 0


for t in range(train_size):
    for i in range(train_size):
        if np.random.rand() < epsilon:

            action = np.random.choice(n)
        else:

            action = np.argmax(action_values[i])

        reward = train_d2d_cache_hits_per_user[i]


        action_counts[i, action] += 1
        action_values[i, action] += (reward - action_values[i, action]) / action_counts[i, action]

        total_rewards += reward


        regret[t, i] = optimal_action_values[i] - reward

predicted_rewards = np.zeros_like(test_d2d_cache_hits_per_user)

for t in range(test_d2d_cache_hits_per_user.shape[0]):
    for i in range(train_size, M):
        action = np.argmax(action_values[i])


        predicted_rewards[t] = action_values[i, action]


predicted_loss_contextual = np.mean(np.square(test_d2d_cache_hits_per_user - predicted_rewards))

print("Loss for Predicted Rewards (Contextual Bandits):", predicted_loss_contextual)


Loss for Predicted Rewards (Contextual Bandits): 2658.5


In [ ]:
predicted_rewards

array([0, 0])

In [ ]:
test_d2d_cache_hits_per_user

array([49, 54])